<a href="https://colab.research.google.com/github/atwine/GCN/blob/master/Implementation_of_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graph Convolution Network

These are used to help work with data which has not euclidian distances, such as data that deals with protein structures and also network relationships

## Clone the necessary lib

The supporting codes used in the notebook are from GitHub as seen below.

A special thanks to the people that worked on them to help make this process easy for me. I am a learner so I just did a replication of the experiment so I can understand some of the processes involved.

In [0]:
import os  #to play around with folders

In [6]:
#Install Kegra
!git clone https://github.com/Awannaphasch2016/kegra_gcn.git

Cloning into 'kegra_gcn'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 30 (delta 1), reused 30 (delta 1), pack-reused 0
Unpacking objects: 100% (30/30), done.


In [0]:
os.chdir('kegra_gcn')

In [8]:
!python setup.py install

running install
running bdist_egg
running egg_info
creating kegra.egg-info
writing kegra.egg-info/PKG-INFO
writing dependency_links to kegra.egg-info/dependency_links.txt
writing requirements to kegra.egg-info/requires.txt
writing top-level names to kegra.egg-info/top_level.txt
writing manifest file 'kegra.egg-info/SOURCES.txt'
writing manifest file 'kegra.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/kegra
copying kegra/anak.py -> build/lib/kegra
copying kegra/__init__.py -> build/lib/kegra
copying kegra/utils.py -> build/lib/kegra
copying kegra/draw_graph.py -> build/lib/kegra
copying kegra/train.py -> build/lib/kegra
copying kegra/random_code.py -> build/lib/kegra
creating build/lib/kegra/layers
copying kegra/layers/graph.py -> build/lib/kegra/layers
copying kegra/layers/__init__.py -> build/lib/kegra/layers
creating build/lib/kegra/testing
copying kegra/testing/

In [0]:
os.chdir('/content/kegra_gcn/GCN-tutorial')

In [14]:
#this must be in the kegra folder otherwise there will be errors loading the dataset needed.
!git clone https://github.com/C-opt/GCN-tutorial.git

Cloning into 'GCN-tutorial'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 30 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (30/30), done.


# Notebook Implementation

Here I implement the notebook I learnt from, I haven't altered the codes at all, I will do in due course as I better understand the concept of GCN


In [0]:
from __future__ import print_function

from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from kegra.layers.graph import GraphConvolution
import utils

import time

In [0]:
DATASET = 'cora'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 500
PATIENCE = 10  # early stopping patience

In [19]:
X, A, y = utils.load_data(dataset=DATASET)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


Split dataset into train, validation, and test batches. The focus here is to split only the labels, not X or A itself. We can sample the training data in X and A by train_mask.

In [0]:
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = utils.get_splits(y)

In [21]:

print("X dim={}".format(X.shape))
print("A dim={}".format(A.shape))
print("y dim={}".format(y.shape))

print("y_train dim={}".format(y_train.shape))
print("y_val dim={}".format(y_val.shape))
print("y_test dim={}".format(y_test.shape))

tmp_sum = 0
for i in range(train_mask.shape[0]):
    if train_mask[i] == True:
        tmp_sum+=1

print(tmp_sum)
print(train_mask)

X dim=(2708, 1433)
A dim=(2708, 2708)
y dim=(2708, 7)
y_train dim=(2708, 7)
y_val dim=(2708, 7)
y_test dim=(2708, 7)
80
[ True  True  True ... False False False]


In [0]:
# Normalize X
from numpy import linalg as LA
for i in range(X.shape[0]):
    X[i] /= LA.norm(X[i])

Take a peek at the 100-th one-hot embedded paper after performing normalization

In [23]:
utils.look_sparse_matrix(X, 100)

#from what i understand in the information below, all the other indexes are linked to the paper 0 in the graph which makes this really interesting

62:0.20000000298023224
99:0.20000000298023224
132:0.20000000298023224
142:0.20000000298023224
292:0.20000000298023224
402:0.20000000298023224
462:0.20000000298023224
495:0.20000000298023224
507:0.20000000298023224
575:0.20000000298023224
648:0.20000000298023224
675:0.20000000298023224
724:0.20000000298023224
733:0.20000000298023224
778:0.20000000298023224
779:0.20000000298023224
821:0.20000000298023224
1071:0.20000000298023224
1097:0.20000000298023224
1151:0.20000000298023224
1230:0.20000000298023224
1331:0.20000000298023224
1334:0.20000000298023224
1348:0.20000000298023224
1422:0.20000000298023224
tmp_sum=[[1.]]==1


In [24]:
if FILTER == 'localpool':
    """ Local pooling filters (see 'renormalization trick' in Kipf & Welling, arXiv 2016) """
    print('Using local pooling filters...')
    A_hat = utils.preprocess_adj(A, SYM_NORM)
    support = 1
    graph = [X, A_hat]
elif FILTER == 'chebyshev':
    """ Chebyshev polynomial basis filters (Defferard et al., NIPS 2016)  """
    print('Using Chebyshev polynomial basis filters...')
    L = normalized_laplacian(A, SYM_NORM)
    L_scaled = rescale_laplacian(L)
    T_k = chebyshev_polynomial(L_scaled, MAX_DEGREE)
    support = MAX_DEGREE + 1
    graph = [X]+T_k
    
else:
    raise Exception('Invalid filter type.')

Using local pooling filters...


# Modeling done here

This is where the magic happens

In [0]:
#Define input tensor
A_hat_in = Input(shape=A_hat.shape, batch_shape=(None, None), sparse=True)
X_in = Input(shape=(X.shape[1],))

# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H_1 = Dropout(0.5)(X_in)
H_2 = GraphConvolution(16, support, activation='relu', kernel_regularizer=l2(5e-4))([H_1]+[A_hat_in])
H_2 = Dropout(0.5)(H_2)
Y_out = GraphConvolution(y.shape[1], support, activation='softmax')([H_2]+[A_hat_in]) #H_{l+1} = f_act(H_l, A)

# Compile model
model = Model(inputs=[X_in]+[A_hat_in], outputs=Y_out)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

In [32]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_4[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_3 (GraphConvo (None, 16)           22944       dropout_3[0][0]                  
                                                                 input_3[0][0]              

In [0]:
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

In [34]:
# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])
    
    # Train / validation scores
    train_val_loss, train_val_acc = utils.evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

# Testing
test_loss, test_acc = utils.evaluate_preds(preds, [y_test], [idx_test])
print("Test set results:",
      "loss= {:.4f}".format(test_loss[0]),
      "accuracy= {:.4f}".format(test_acc[0]))

Epoch: 0001 train_loss= 1.9320 train_acc= 0.3000 val_loss= 1.9381 val_acc= 0.2024 time= 0.6628
Epoch: 0002 train_loss= 1.9102 train_acc= 0.3875 val_loss= 1.9255 val_acc= 0.1976 time= 0.1261
Epoch: 0003 train_loss= 1.8829 train_acc= 0.4250 val_loss= 1.9107 val_acc= 0.2214 time= 0.1332
Epoch: 0004 train_loss= 1.8539 train_acc= 0.4500 val_loss= 1.8956 val_acc= 0.2357 time= 0.1250
Epoch: 0005 train_loss= 1.8225 train_acc= 0.5000 val_loss= 1.8789 val_acc= 0.2738 time= 0.1271
Epoch: 0006 train_loss= 1.7883 train_acc= 0.6250 val_loss= 1.8608 val_acc= 0.3429 time= 0.1287
Epoch: 0007 train_loss= 1.7518 train_acc= 0.6875 val_loss= 1.8412 val_acc= 0.4214 time= 0.1304
Epoch: 0008 train_loss= 1.7129 train_acc= 0.7500 val_loss= 1.8195 val_acc= 0.4810 time= 0.1364
Epoch: 0009 train_loss= 1.6721 train_acc= 0.7875 val_loss= 1.7970 val_acc= 0.5310 time= 0.1269
Epoch: 0010 train_loss= 1.6291 train_acc= 0.8250 val_loss= 1.7741 val_acc= 0.5857 time= 0.1281
Epoch: 0011 train_loss= 1.5843 train_acc= 0.8375 v